# Starter Tutorial
This tutorial demonstrates basic steps for conducting an AI-powered survey using EDSL. 

## Prerequisites
Before running the code below, ensure that you have already completed technical setup:

- Download the EDSL package. See [Installation](https://docs.expectedparrot.com/en/latest/installation.html) instructions.

- Create a [Coop](https://docs.expectedparrot.com/en/latest/coop.html) account and activate [Remote Inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) or store your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you plan to use locally.

If you encounter any issues or have questions, please email us at info@expectedparrot.com or post a question at our [Discord channel](https://discord.com/invite/mxAYkjfy9m).

## Conducting an AI-powered survey
In the steps below we show how to create and run a simple question in EDSL. 
Then we show how to design a more complex survey with AI agents and different language models.

### Run a simple question
In this first example we create a simple multiple choice question, administer it to a language model and inspect the response:

In [1]:
# Import a question type
from edsl import QuestionMultipleChoice

# Construct a question in the question type template
q = QuestionMultipleChoice(
    question_name = "example_question",
    question_text = "How do you feel today?",
    question_options = ["Bad", "OK", "Good"]
)

# Run it with the default language model
results = q.run()

# Inspect the results
results.select("example_question").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━┓
┃ answer            ┃
┃ .example_question ┃
┡━━━━━━━━━━━━━━━━━━━┩
│ Good              │
└───────────────────┘

*Note:* The default language model at the time this notebook was last updated was gpt-4o; you will need an API key for OpenAI to use this model and run this example locally.
See instructions on storing your [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html). 
Alternatively, you can activate [Remote Inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) at your [Coop](https://docs.expectedparrot.com/en/latest/coop.html) account to run the example on the Expected Parrot server.

### Conduct a survey with agents and models
In this next example we create a survey of multiple questions, design personas for AI agents to answer the questions and select the language models that we want to generate the responses.
We also show how to parameterize questions with context or data to administer different versions of the questions efficiently.
This can be useful for comparing responses for different data, agents and models, and performing data labeling tasks.

We also show how to filter, sort, select and print components of the dataset of results.

#### Question types
To see examples of all EDSL question types, run:

In [2]:
from edsl import Question

Question.available()

['checkbox',
 'extract',
 'free_text',
 'functional',
 'likert_five',
 'linear_scale',
 'list',
 'multiple_choice',
 'numerical',
 'rank',
 'top_k',
 'yes_no']

#### Language models
Newly released language models are automatically added to EDSL when they become available. 
To see a current list of available models, run:

In [3]:
from edsl import Model

# Model.available() # uncomment this line and run it

To confirm the current default model:

In [4]:
# Model() # uncomment this line and run it

#### Example survey

In [5]:
# Import question types and survey components
from edsl import (
    QuestionLinearScale, QuestionFreeText, Survey,
    ScenarioList, Scenario, 
    AgentList, Agent, 
    ModelList, Model,
    Cache
)

# Construct questions
q1 = QuestionLinearScale(
    question_name = "enjoy",
    question_text = "On a scale from 1 to 5, how much do you enjoy {{ activity }}?",
    question_options = [1,2,3,4,5],
    option_labels = {1:"Not at all", 5:"Very much"}
)

q2 = QuestionFreeText(
    question_name = "recent",
    question_text = "Describe the most recent time you were {{ activity }}."
)

# Combine questions in a survey
survey = Survey(questions = [q1, q2])

# Add data to questions using scenarios
activities = ["exercising", "reading", "cooking"]

scenarios = ScenarioList(
    Scenario({"activity": a}) for a in activities
)

# Create personas for AI agents to answer the questions
personas = ["athlete", "student", "chef"]

agents = AgentList(
    Agent(traits = {"persona": p}) for p in personas
)

# Select language models to generate responses
models = ModelList(
    Model(m) for m in ["gpt-4o", "claude-3-5-sonnet-20240620"]
)

# Run the survey with the scenarios, agents and models
results = survey.by(scenarios).by(agents).by(models).run()

# Filter, sort, select and print components of the results to inspect
(
    results
    .filter("activity == 'reading' and persona == 'chef'")
    .sort_by("model")
    .select("model", "activity", "persona", "answer.*")
    .print(format="rich",
            pretty_labels = ({
                "model.model":"Model",
                "scenario.activity":"Activity",
                "agent.persona":"Agent persona",
                "answer.enjoy":"Enjoy",
                "answer.recent":"Recent"
            })
        )
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Model                      ┃ Activity ┃ Agent persona ┃ Recent                                          ┃ Enjoy ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ claude-3-5-sonnet-20240620 │ reading  │ chef          │ As a chef, I'm always reading cookbooks,        │ 3     │
│                            │          │               │ culinary magazines, and food blogs to stay      │       │
│                            │          │               │ up-to-date with the latest trends and           │       │
│                            │          │               │ techniques in the culinary world. Just          │       │
│                            │          │               │ yesterday, I was poring over a new cookbook     │       │
│                            │          │               │ featuring modern interpretations of classic     │       │
│                            │          │               │ French cuisine. I was particularly interested   │       │
│                            │          │               │ in a section on innovative sauces and           │       │
│                            │          │               │ reductions. Reading about food is an essential  │       │
│                            │          │               │ part of my professional development and helps   │       │
│                            │          │               │ inspire new ideas for my own cooking. It's not  │       │
│                            │          │               │ just about recipes, but also understanding      │       │
│                            │          │               │ flavor combinations, plating techniques, and    │       │
│                            │          │               │ the cultural significance of different dishes.  │       │
├────────────────────────────┼──────────┼───────────────┼─────────────────────────────────────────────────┼───────┤
│ gpt-4o                     │ reading  │ chef          │ The most recent time I was reading, I was       │ 4     │
│                            │          │               │ flipping through a cookbook that focuses on     │       │
│                            │          │               │ Mediterranean cuisine. I was particularly       │       │
│                            │          │               │ interested in a recipe for a traditional Greek  │       │
│                            │          │               │ moussaka. The layers of eggplant, seasoned      │       │
│                            │          │               │ ground meat, and creamy béchamel sauce sounded  │       │
│                            │          │               │ divine. I was taking notes on the different     │       │
│                            │          │               │ spices and techniques used, thinking about how  │       │
│                            │          │               │ I could incorporate some of those flavors into  │       │
│                            │          │               │ my own dishes. It was a delightful and          │       │
│                            │          │               │ inspiring read!                                 │       │
└────────────────────────────┴──────────┴───────────────┴─────────────────────────────────────────────────┴───────┘

### Exploring your results
EDSL comes with built-in methods for analyzing and visualizing your results. 
For example, you can access results as a Pandas dataframe:

In [6]:
# Convert the Results object to a pandas dataframe
(
    results
     .sort_by("model", "activity", "persona")
     .select("model", "activity", "persona", "recent", "enjoy")
     .to_pandas(remove_prefix=True)
)

,model,activity,persona,recent,enjoy
0,claude-3-5-sonnet-20240620,cooking,athlete,"I apologize, but as an athlete, I don't typica...",2
1,claude-3-5-sonnet-20240620,cooking,chef,"As a chef, I'm always cooking, but let me tell...",5
2,claude-3-5-sonnet-20240620,cooking,student,"As a student, I recently cooked a simple meal ...",3
3,claude-3-5-sonnet-20240620,exercising,athlete,"As an athlete, I'm constantly training and exe...",5
4,claude-3-5-sonnet-20240620,exercising,chef,"I apologize, but as a chef, I don't have perso...",3
5,claude-3-5-sonnet-20240620,exercising,student,"As a student, the most recent time I exercised...",3
6,claude-3-5-sonnet-20240620,reading,athlete,"As an athlete, I don't spend a ton of time rea...",2
7,claude-3-5-sonnet-20240620,reading,chef,"As a chef, I'm always reading cookbooks, culin...",3
8,claude-3-5-sonnet-20240620,reading,student,"As a student, I was actually reading just last...",4
9,gpt-4o,cooking,athlete,"Sure! The most recent time I was cooking, I de...",3


The `columns` method will display a list of all the components of your results, which you can then `select` and `print` to show them:

In [7]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'agent.persona',
 'answer.enjoy',
 'answer.recent',
 'comment.enjoy_comment',
 'comment.recent_comment',
 'generated_tokens.enjoy_generated_tokens',
 'generated_tokens.recent_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.enjoy_system_prompt',
 'prompt.enjoy_user_prompt',
 'prompt.recent_system_prompt',
 'prompt.recent_user_prompt',
 'question_options.enjoy_question_options',
 'question_options.recent_question_options',
 'question_text.enjoy_question_text',
 'question_text.recent_question_text',
 'question_type.enjoy_question_type',
 'question_type.recent_question_type',
 'raw_model_response.enjoy_cost',
 'raw_model_response.enjoy_one_usd_buys',
 'raw_model_response.enjoy_raw_model_response',
 'raw_model_response.recent_cost',
 'raw_model_response.recent_one_usd_

The `Results` object also supports SQL-like queries:

In [8]:
# Execute an SQL-like query on the results
results.sql("""
select model, activity, persona, recent, enjoy 
from self
order by 1,2,3
""", shape="wide")

,model,activity,persona,recent,enjoy
0,claude-3-5-sonnet-20240620,cooking,athlete,"I apologize, but as an athlete, I don't typica...",2
1,claude-3-5-sonnet-20240620,cooking,chef,"As a chef, I'm always cooking, but let me tell...",5
2,claude-3-5-sonnet-20240620,cooking,student,"As a student, I recently cooked a simple meal ...",3
3,claude-3-5-sonnet-20240620,exercising,athlete,"As an athlete, I'm constantly training and exe...",5
4,claude-3-5-sonnet-20240620,exercising,chef,"I apologize, but as a chef, I don't have perso...",3
5,claude-3-5-sonnet-20240620,exercising,student,"As a student, the most recent time I exercised...",3
6,claude-3-5-sonnet-20240620,reading,athlete,"As an athlete, I don't spend a ton of time rea...",2
7,claude-3-5-sonnet-20240620,reading,chef,"As a chef, I'm always reading cookbooks, culin...",3
8,claude-3-5-sonnet-20240620,reading,student,"As a student, I was actually reading just last...",4
9,gpt-4o,cooking,athlete,"Sure! The most recent time I was cooking, I de...",3


Learn more about working with results in the [Results](https://docs.expectedparrot.com/en/latest/results.html) section.

## Posting content at the Coop
To post an object (here, our results):

In [9]:
results.push(visibility="public")

{'description': None,
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/f674ba78-17d5-4628-9b57-ec7c5a96718c',
 'uuid': 'f674ba78-17d5-4628-9b57-ec7c5a96718c',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

Posting this notebook:

In [10]:
from edsl import Notebook

notebook = Notebook(path="starter_tutorial.ipynb")

notebook.push(description="Starter Tutorial", visibility="public")

{'description': 'Starter Tutorial',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/d11a525e-d454-4eb1-bd96-0ab9d771249e',
 'uuid': 'd11a525e-d454-4eb1-bd96-0ab9d771249e',
 'version': '0.1.33.dev1',
 'visibility': 'public'}